## Import libraries

In [9]:
import re
import nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem  import WordNetLemmatizer
from nltk.stem import PorterStemmer
import contractions
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [10]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('brown')

[nltk_data] Downloading package stopwords to /home/anish/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/anish/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/anish/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /home/anish/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package brown to /home/anish/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


True

### Import Dataset

In [14]:
data_df = pd.read_csv('../data/raw/leave_reasons.csv')

### Text Processing

In [15]:
stop_word = list(set(stopwords.words('english')))

In [16]:
stop_word = list(filter(lambda x : x!=str('not'),stop_word))

In [17]:
lemmatizer = WordNetLemmatizer()

In [18]:
def text_processing(text):
    
    #removing everything except letter and space
    temp = re.sub(r"[^A-Za-z ]+",r"",text)
    #lower case
    temp = temp.lower()
    #remove unnecessary spaces
    temp = " ".join(temp.split())
    #word tokenize
    temp = word_tokenize(temp)
    # remove stop words
    temp = [ item for item in temp if item not in stop_word]
    #lemmatize
    temp = [lemmatizer.lemmatize(item) for item in temp]
    return temp

In [19]:
data_df['token'] = data_df['reason'].map(text_processing)

In [20]:
def noun_phrase(token):
  key = pos_tag(token)
  data=[]
  for tag in key:
    if tag[1] == 'NN':
      data.append(tag[0])
  if len(data) == 0:
    return token
  else:
    return list(data)

In [21]:
data_df['noun_phrase'] = data_df['token'].map(noun_phrase)

In [22]:
data_df['len'] = data_df['noun_phrase'].map(len)

In [23]:
def replace_word(array):
  text = " ".join(array)
  text = text.replace('bandha', 'strike')
  text = text.replace('banda','strike')
  text = text.replace('bandh', 'strike')
  return text.split()

In [25]:
data_df['noun_phrase'] = data_df['noun_phrase'].map(replace_word)

In [26]:
filehandler = open(b"../data/processed/processed_text","wb")
pickle.dump(data_df,filehandler)